In [2]:
!pip install -q trino

In [8]:
# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'server' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://server:8181/catalog"
TRINO_URI = "http://trino:8080"
WAREHOUSE = "demo"

# Create Trino Catalog

In [9]:
from trino.dbapi import connect

conn = connect(host=TRINO_URI, user="trino")

In [10]:
cur = conn.cursor()
cur.execute(
    f"""
    CREATE CATALOG lakekeeper USING iceberg
    WITH (
        "iceberg.catalog.type" = 'rest',
        "iceberg.rest-catalog.uri" = '{CATALOG_URL}',
        "iceberg.rest-catalog.warehouse" = '{WAREHOUSE}',
        "iceberg.rest-catalog.security" = 'NONE',
        "iceberg.rest-catalog.vended-credentials-enabled" = 'true',
        "s3.region"= 'dummy',
        "s3.path-style-access" = 'true',
        "s3.endpoint" = 'http://minio:9000',
        "fs.native-s3.enabled" = 'true'
    )
"""
)

## Read and Write Tables

In [ ]:
# Connect directly to demo catalog, so that we don't have to use it as a prefix
conn = connect(host=TRINO_URI, user="trino", catalog="lakekeeper")
cur = conn.cursor()

In [ ]:
cur.execute("CREATE SCHEMA IF NOT EXISTS trino_namespace")

In [ ]:
cur.execute(
    "CREATE TABLE trino_namespace.my_table (my_ints INT, my_floats DOUBLE, strings VARCHAR) WITH (format='PARQUET')"
)
cur.execute(
    "INSERT INTO trino_namespace.my_table VALUES (1, 1.0, 'a'), (2, 2.0, 'b')"
)

In [ ]:
cur.execute("SELECT * FROM trino_namespace.my_table").fetchall()

In [16]:
cur.execute("use lakekeeper.my_db").fetchall()

[]

In [17]:
cur.execute("show tables").fetchall()

[['stations'], ['users']]

In [18]:
cur.execute("select * from users").fetchall()

TrinoQueryError: TrinoQueryError(type=INTERNAL_ERROR, name=GENERIC_INTERNAL_ERROR, message="io.trino.spi.TrinoException: Error processing metadata for table my_db.users", query_id=20250331_150946_00008_cctnw)